### Machine Learning Advanced Big data Project

In [0]:
orders_sdf = spark.read.csv('/FileStore/tables/orders.csv', header=True, inferSchema=True)
trains_sdf = spark.read.csv('/FileStore/tables/order_products_train.csv', header=True, inferSchema=True)
products_sdf = spark.read.csv('/FileStore/tables/products.csv', header=True, inferSchema=True)
aisles_sdf = spark.read.csv('/FileStore/tables/aisles.csv', header=True, inferSchema=True)
depts_sdf = spark.read.csv('/FileStore/tables/departments.csv', header=True, inferSchema=True)

In [0]:
%fs 
cp /FileStore/tables/order_products_prior.zip file:/home/order_products_prior.zip 

res3: Boolean = true

In [0]:
import pandas as pd

priors_pdf = pd.read_csv('/home/order_products_prior.zip', compression='zip', header=0, sep=',', quotechar='"')
priors_sdf = spark.createDataFrame(priors_pdf)
del priors_pdf # 메모리 절약을 위해 pandas dataframe삭제

In [0]:
orders_sdf.createOrReplaceTempView("orders")
priors_sdf.createOrReplaceTempView("priors")
trains_sdf.createOrReplaceTempView("trains")
products_sdf.createOrReplaceTempView("products")
aisles_sdf.createOrReplaceTempView("aisles")
depts_sdf.createOrReplaceTempView("depts")

In [0]:
#테이블 등록
spark.catalog.listTables()

Out[8]: [Table(name='order_priors_prods', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='user_mart_01', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='aisles', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='depts', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='orders', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='priors', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='products', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='trains', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/order_priors_prods

res4: Boolean = true

In [0]:
%sql 
drop table if exists order_priors_prods; 

-- priors와 orders를 조인 
-- orders에서는 pk를 확인할 수 없기 때문에 조인
create table  order_priors_prods 
as 
select a.order_id, a.product_id, a.add_to_cart_order, a.reordered, b.user_id, b.eval_set, b.order_number, b.order_dow, b.order_hour_of_day, b.days_since_prior_order 
from priors a, orders b 
where a.order_id = b.order_id;

num_affected_rows,num_inserted_rows


### Creating a product analysis table based on product level analysis attributes
* PK is a product code (product_id) and generates a product analysis table with attributes analyzed in the previous EDA..

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/prd_mart

res6: Boolean = true

In [0]:
%sql
drop table if exists prd_mart;

create table prd_mart
as
with 
-- with 구문 첫번째 집합. product_id 레벨로 group by 하여 상품별 서로 다른 개별 사용자 비율을 추출한 결과에 상품명과 상품 중분류명 알기 위해 products와 aisles로 조인
order_prods_grp as
(
  select a.product_id 
    -- ## 상품별 재주문 속성
    , sum(case when reordered=1 then 1 else 0 end) as prd_reordered_cnt -- 상품별 재 주문 건수
    , sum(case when reordered=0 then 1 else 0 end) as prd_no_reordered_cnt -- 상품별 재 주문 하지 않은 건수 
    , avg(reordered) prd_avg_reordered -- 상품별 재 주문 비율
    -- ## 상품별 고유 사용자 및 이전 주문이후 걸린 일자 속성. 
    , count(distinct user_id) prd_unq_usr_cnt -- 상품별 고유 사용자 건수
    , count(*)  prd_total_cnt -- 상품별 건수
    , count(distinct user_id)/count(*) as prd_usr_ratio -- 상품별 전체 건수 대비 고유 사용자 비율
    , max(c.aisle_id) aisle_id -- 상품 중분류 코드 
    , nvl(avg(days_since_prior_order), 0) as prd_avg_prior_days -- 평균 이전 주문이후 걸린 일자, null인 경우 0으로 변환. 
    , nvl(min(days_since_prior_order), 0) as prd_min_prior_days -- 최소 이전 주문이후 걸린 일자, null인 경우 0으로 변환. 
    , nvl(max(days_since_prior_order), 0) as prd_max_prior_days -- 최대 이전 주문이후 걸린 일자, null인 경우 0으로 변환. 
    from order_priors_prods a, products b, aisles c
  where a.product_id = b.product_id 
  and b.aisle_id = c.aisle_id
  group by a.product_id
),
-- with 구문 두번째 집합. product_id 레벨로 group by 하여 상품별 서로 다른 개별 사용자 비율을 추출한 결과에 product_name과 중분류명, 대분류명을 알기 위해 aisles와 dept로 조인. 
order_aisles_grp as
(
  select c.aisle_id as aisle_id 
     , count(distinct a.user_id) aisle_distinct_usr_cnt -- 상품 중분류별 고유 사용자 건수
     , count(*)  aisle_total_cnt -- 상품 중분류 건수
     , count(distinct a.user_id)/count(*) as aisle_usr_ratio -- 상품 중분류 건수 대비 고유 사용자 건수 비율
  from order_priors_prods a, products b, aisles c
  where a.product_id = b.product_id 
  and b.aisle_id = c.aisle_id
  group by c.aisle_id
),
-- with 구문 세번째 집합. 상품 중분류 별 개별 사용자 비율과 상품별 개별 사용자 비율 차이 추출. 
order_prd_grp_aisle as
(
  select product_id, prd_reordered_cnt,  prd_no_reordered_cnt, prd_avg_reordered, prd_unq_usr_cnt, prd_total_cnt, prd_usr_ratio
    , prd_avg_prior_days, prd_min_prior_days, prd_max_prior_days-- 상품별 속성들
    , b.aisle_distinct_usr_cnt, b.aisle_total_cnt, b.aisle_usr_ratio -- 상품 중분류별 속성들 
    , a.prd_usr_ratio - b.aisle_usr_ratio as usr_ratio_diff -- 상품 중분류 별 개별 사용자 비율과 상품별 개별 사용자 비율 차이
  from order_prods_grp a, order_aisles_grp b
  where a.aisle_id = b.aisle_id
) 
-- end of with 절
select * from order_prd_grp_aisle

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from prd_mart limit 10

product_id,prd_reordered_cnt,prd_no_reordered_cnt,prd_avg_reordered,prd_unq_usr_cnt,prd_total_cnt,prd_usr_ratio,prd_avg_prior_days,prd_min_prior_days,prd_max_prior_days,aisle_distinct_usr_cnt,aisle_total_cnt,aisle_usr_ratio,usr_ratio_diff
47908,0,3,0.0,3,3,1.0,22.333333333333332,7.0,30.0,85357,575881,0.14821985792203599,0.851780142077964
9856,0,3,0.0,3,3,1.0,10.666666666666666,7.0,17.0,85357,575881,0.14821985792203599,0.851780142077964
3832,0,2,0.0,2,2,1.0,21.5,13.0,30.0,85357,575881,0.14821985792203599,0.851780142077964
12120,0,3,0.0,3,3,1.0,6.333333333333333,6.0,7.0,85357,575881,0.14821985792203599,0.851780142077964
10536,3,7,0.3,7,10,0.7,21.625,9.0,30.0,85357,575881,0.14821985792203599,0.551780142077964
33171,0,8,0.0,8,8,1.0,10.571428571428571,3.0,30.0,85357,575881,0.14821985792203599,0.851780142077964
29994,13,4,0.7647058823529411,4,17,0.23529411764705882,7.666666666666667,1.0,21.0,85357,575881,0.14821985792203599,0.08707425972502283
28551,28,7,0.8,7,35,0.2,6.8,0.0,29.0,85357,575881,0.14821985792203599,0.051780142077964025
46860,0,5,0.0,5,5,1.0,11.25,3.0,22.0,85357,575881,0.14821985792203599,0.851780142077964
1431,9,14,0.391304347826087,14,23,0.6086956521739131,11.428571428571429,1.0,30.0,85357,575881,0.14821985792203599,0.4604757942518771


In [0]:
%sql
--49676
select count(*) from prd_mart

count(1)
49676


In [0]:
import pyspark.sql.functions as F

prd_mart_sdf = spark.sql("select * from prd_mart")

display(prd_mart_sdf.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in prd_mart_sdf.columns]))

product_id,prd_reordered_cnt,prd_no_reordered_cnt,prd_avg_reordered,prd_unq_usr_cnt,prd_total_cnt,prd_usr_ratio,prd_avg_prior_days,prd_min_prior_days,prd_max_prior_days,aisle_distinct_usr_cnt,aisle_total_cnt,aisle_usr_ratio,usr_ratio_diff
0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Create user analysis tables based on user-level analysis properties
* PK is the user ID (user_id) and creates a user analysis table with attributes analyzed by previous EDA.
* Order_id is required to create prediction data in the future. To this end, it is necessary to extract order_id by joining the order data for training and testing with user_id.
* The order table is user_id level m, but if eval_set is train and test, the user_id level is 1, so the user_mart table level does not change when joining.

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/user_mart_01

res7: Boolean = false

In [0]:
%sql
drop table if exists user_mart_01;

create table user_mart_01
as
select user_id 
  , count(*) as usr_total_cnt -- 사용자별 주문 건수
  -- 주문 건수 관련 속성 추출. 
  , count(distinct product_id) prd_uq_cnt  -- 사용자별 고유 상품 주문 건수
  , count(distinct order_id) order_uq_cnt -- 사용자별 고유 주문 건수
  , count(*)/count(distinct order_id) as usr_avg_prd_cnt -- 사용자별 1회 주문시 평균 주문 상품 건수
  , count(*)/count(distinct product_id) as usr_avg_uq_prd_cnt -- 사용자별 1회 주문시 평균 고유 주문 상품 건수
  , count(distinct product_id)/count(*) as usr_uq_prd_ratio --사용자별 총 상품 건수 대비 고유 상품 건수 비율
  -- ### reordered 관련 속성 추출. ###
  , sum(reordered) usr_reord_cnt -- 사용자별 reordered된 상품 건수
  , sum(case when reordered = 0 then 1 else 0 end) as usr_no_reord_cnt -- 사용자별 reorder 하지 않은 상품 건수. count(*) - sum(reoredred)와 동일. 
  , avg(reordered) usr_reordered_avg -- 사용자별 reordered 비율
  -- ### days_since_prior_order 관련 속성 추출. ###
  , avg(days_since_prior_order) usr_avg_prior_days
  , max(days_since_prior_order) usr_max_prior_days
  , min(days_since_prior_order) usr_min_prior_days
  -- ### order_dow, order_hour_of_day 관련 속성 추출. ###
  , avg(order_dow) usr_avg_order_dow
  , avg(order_hour_of_day) usr_avg_order_hour_of_day
  -- 사용자별 최대 order_number
  , max(order_number) as usr_max_order_number
from order_priors_prods a group by user_id

num_affected_rows,num_inserted_rows


In [0]:
%sql 
select count(*) from user_mart_01

count(1)
206209


In [0]:
%sql
drop table if exists user_mart;
-- cmd 20 
-- orders는 eval_set이 train/test일 경우 한개의 user_id가 한개의 order_id를 가짐. 때문에 train/test인 경우 조인키값 user_id로 1레벨이 됨.
-- order_priors_prods에 있는 모든 user_id는 orders의 모든 user_id와 동일. orders는 user_id별로 여러건의 order가 있고, 이들중 마지막 order를 train또는 test로 할당하기 때문
-- 따라서 user_mart_01과 eval_set이 train과 test인 orders를 user_id로 조인하면 1:1 조인이 되고 user_mart_01의 집합 레벨의 변화가 없음. outer join을 하지 않아도 됨. 
create table user_mart
as
select a.*, b.order_id, b.eval_set, b.days_since_prior_order
from user_mart_01 a, orders b
where a.user_id = b.user_id
and b.eval_set in ('train', 'test')   

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from user_mart limit 10

user_id,usr_total_cnt,prd_uq_cnt,order_uq_cnt,usr_avg_prd_cnt,usr_avg_uq_prd_cnt,usr_uq_prd_ratio,usr_reord_cnt,usr_no_reord_cnt,usr_reordered_avg,usr_avg_prior_days,usr_max_prior_days,usr_min_prior_days,usr_avg_order_dow,usr_avg_order_hour_of_day,usr_max_order_number,order_id,eval_set,days_since_prior_order
1,59,18,10,5.9,3.2777777777777777,0.3050847457627119,41,18,0.6949152542372882,20.25925925925926,30.0,0.0,2.6440677966101696,10.542372881355933,10,1187899,train,14.0
2,195,102,14,13.928571428571429,1.911764705882353,0.5230769230769231,93,102,0.47692307692307695,15.967032967032967,30.0,3.0,2.005128205128205,10.441025641025641,14,1492625,train,30.0
3,88,33,12,7.333333333333333,2.6666666666666665,0.375,55,33,0.625,11.487179487179487,21.0,7.0,1.0113636363636365,16.352272727272727,12,2774568,test,11.0
4,18,17,5,3.6,1.0588235294117647,0.9444444444444444,1,17,0.05555555555555555,15.357142857142858,21.0,0.0,4.722222222222222,13.11111111111111,5,329954,test,30.0
5,37,23,4,9.25,1.608695652173913,0.6216216216216216,14,23,0.3783783783783784,14.5,19.0,10.0,1.6216216216216217,15.72972972972973,4,2196797,train,6.0
6,14,12,3,4.666666666666667,1.1666666666666667,0.8571428571428571,2,12,0.14285714285714285,7.8,12.0,6.0,3.857142857142857,17.0,3,1528013,test,22.0
7,206,68,20,10.3,3.0294117647058822,0.3300970873786408,138,68,0.6699029126213593,13.54639175257732,30.0,2.0,1.7281553398058251,13.631067961165048,20,525192,train,6.0
8,49,36,3,16.333333333333332,1.3611111111111112,0.7346938775510204,13,36,0.2653061224489796,30.0,30.0,30.0,4.204081632653061,2.4489795918367347,3,880375,train,10.0
9,76,58,3,25.333333333333332,1.3103448275862069,0.7631578947368421,18,58,0.23684210526315788,24.26086956521739,30.0,6.0,2.6973684210526314,14.263157894736842,3,1094988,train,30.0
10,143,94,5,28.6,1.5212765957446808,0.6573426573426573,49,94,0.34265734265734266,20.746376811594203,30.0,12.0,4.013986013986014,16.902097902097903,5,1822501,train,30.0


In [0]:
%sql
--206209
select count(*) from user_mart

count(1)
206209


In [0]:
%sql
select count(*)
from orders b
where b.eval_set in ('train', 'test')  

count(1)
206209


user_id : 1 means that the tables includes records for training 

user_id : 3 means that the tables includes records for test

In [0]:

%sql
select * from orders where user_id = 1

order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2539329,1,prior,1,2,8,null
2398795,1,prior,2,3,7,15.0
473747,1,prior,3,3,12,21.0
2254736,1,prior,4,4,7,29.0
431534,1,prior,5,4,15,28.0
3367565,1,prior,6,2,7,19.0
550135,1,prior,7,1,9,20.0
3108588,1,prior,8,1,14,14.0
2295261,1,prior,9,1,16,0.0
2550362,1,prior,10,4,8,30.0


In [0]:
%sql
select * from orders where user_id = 3

order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1374495,3,prior,1,1,14,null
444309,3,prior,2,3,19,9.0
3002854,3,prior,3,3,16,21.0
2037211,3,prior,4,2,18,20.0
2710558,3,prior,5,0,17,12.0
1972919,3,prior,6,0,16,7.0
1839752,3,prior,7,0,15,7.0
3225766,3,prior,8,0,17,7.0
3160850,3,prior,9,0,16,7.0
676467,3,prior,10,3,16,17.0


### 사용자 + 상품 레벨의 분석 속성에 기반한 사용자+상품 분석 테이블 생성
* PK 는 사용자아이디(user_id)+상품코드(product_id)이며 이전 EDA에서 분석한 속성들로 사용자+상품 분석 테이블 생성 테이블 생성.
* 앞에서 만든 prd_mart, user_mart를 사용자+상품 분석 테이블과 조인하여 상품관련 속성, 사용자 관련 속성을 결합함.

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/up_mart

res12: Boolean = true

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/up_mart_01

res13: Boolean = false

In [0]:
%sql
select * from order_priors_prods where user_id = 1

order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
473747,196,1,1,1,prior,3,3,12,21.0
473747,12427,2,1,1,prior,3,3,12,21.0
473747,10258,3,1,1,prior,3,3,12,21.0
473747,25133,4,0,1,prior,3,3,12,21.0
473747,30450,5,0,1,prior,3,3,12,21.0
3108588,12427,1,1,1,prior,8,1,14,14.0
3108588,196,2,1,1,prior,8,1,14,14.0
3108588,10258,3,1,1,prior,8,1,14,14.0
3108588,25133,4,1,1,prior,8,1,14,14.0
3108588,46149,5,0,1,prior,8,1,14,14.0


In [0]:
%sql
drop table if exists up_mart;
drop table if exists up_mart_01;

create table up_mart
as
with 
-- 사용자+상품 레벨로 group by 하여 속성 추출. 
up_grp as
(
SELECT user_id, product_id
    , count(*) up_cnt  -- 사용자의 개별 상품별 주문 건수
    , sum(reordered) up_reord_cnt -- 사용자의 개별 상품별 reorder 건수
    , sum(case when reordered=0 then 1 else 0 end) up_no_reord_cnt
    , avg(reordered) up_reoredered_avg -- 사용자의 개별 상품 주문별 reorder비율 
    , max(order_number) up_max_ord_num -- 사용자+상품레벨에서 가장 큰 order_number. order_number는 사용자 별로 주문을 수행한 일련번호를 순차적으로 가짐. 
    , min(order_number) up_min_ord_num -- 사용자+상품레벨에서 가장 작은 order_number
    , avg(add_to_cart_order) up_avg_cart --사용자 상품레벨에서 보통 cart에 몇번째로 담는가?
    , avg(days_since_prior_order) as up_avg_prior_days
    , max(days_since_prior_order) as up_max_prior_days
    , min(days_since_prior_order) as up_min_prior_days
    , avg(order_dow) as up_avg_ord_dow
    , avg(order_hour_of_day) as up_avg_ord_hour
FROM order_priors_prods GROUP BY user_id, product_id
)
-- end of with 절 
-- 사용자 레벨로 group by 한 user_mart 테이블과 조인하여 사용자 레벨 속성과 사용자+상품 레벨 속성의 비율을 추출. 
select a.* 
  , a.up_cnt/b.usr_total_cnt as up_usr_ratio -- 사용자별 전체 주문 건수 대비 사용자+상품 주문 건수 비율
  , a.up_reord_cnt/b.usr_reord_cnt as up_usr_reord_ratio -- 사용자별 전체 재주문 건수 대비 사용자+상품 재주문 건수 비율
  , b.usr_reord_cnt
  , b.usr_max_order_number - a.up_max_ord_num as up_usr_ord_num_diff -- 사용자의 가장 최근 주문(가장 큰 주문번호)에서 현 상품 주문번호가 어느정도 이후에 있는지
from up_grp a, user_mart b
where a.user_id = b.user_id

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: io.delta.exceptions.ProtocolChangedException: The protocol version of the Delta table has been changed by a concurrent update. This happens when multiple writers are writing to an empty directory. Creating the table ahead of time will avoid this conflict. Please try the operation again.
Conflicting commit: {"timestamp":1653751439002,"userId":"6446834719142047","userName":"kjr5189@gmail.com","operation":"CREATE TABLE AS SELECT","operationParameters":{"isManaged":true,"description":null,"partitionBy":[],"properties":{}},"notebook":{"notebookId":"2764109102445363"},"clusterId":"0528-104803-p81cm7gp","isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{"numFiles":"25","numOutputRows":"13307953","numOutputBytes":"299741479"},"engineInfo":"Databricks-Runtime/10.4.x-scala2.12","txnId":"4f981edb-8782-47bf-914d-a96e99d4ee58"}
Refer to https://docs.databricks.com/delta/concurrency-control.html for more details.
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.protocolChangedException(DeltaErrors.scala:1667)
	at com.databricks.sql.transaction.tahoe.ConflictChecker.$anonfun$checkProtocolCompatibility$2(ConflictChecker.scala:156)
	at com.databricks.sql.transaction.tahoe.ConflictChecker.$anonfun$checkProtocolCompatibility$2$adapted(ConflictChecker.scala:154)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.sql.transaction.tahoe.ConflictChecker.checkProtocolCompatibility(ConflictChecker.scala:154)
	at com.databricks.sql.transaction.tahoe.ConflictChecker.checkConflicts(ConflictChecker.scala:115)
	at com.databricks.sql.transaction.tahoe.ConflictCheckerEdge.checkConflicts(ConflictCheckerEdge.scala:202)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImplEdge.checkForConflictsAgainstVersion(OptimisticTransactionImplEdge.scala:507)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImplEdge.checkForConflictsAgainstVersion$(OptimisticTransactionImplEdge.scala:493)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.checkForConflictsAgainstVersion(OptimisticTransaction.scala:104)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.$anonfun$checkForConflicts$4(OptimisticTransaction.scala:1012)
	at scala.runtime.java8.JFunction1$mcVJ$sp.apply(JFunction1$mcVJ$sp.java:23)
	at scala.collection.immutable.NumericRange.foreach(NumericRange.scala:75)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.$anonfun$checkForConflicts$1(OptimisticTransaction.scala:1008)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.withOperationTypeTag(DeltaLogging.scala:156)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.withOperationTypeTag$(DeltaLogging.scala:143)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.withOperationTypeTag(OptimisticTransaction.scala:104)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.$anonfun$recordDeltaOperation$6(DeltaLogging.scala:121)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordFrameProfile(DeltaLogging.scala:171)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordFrameProfile$(DeltaLogging.scala:169)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.recordFrameProfile(OptimisticTransaction.scala:104)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.$anonfun$recordDeltaOperation$5(DeltaLogging.scala:120)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:28)
	at scala.util.DynamicVariable.wit

In [0]:
%sql
--13307953
select count(*) from up_mart

count(1)
13307953


In [0]:
%sql
-- up_mart에서 user_mart로, user_id로 join이 안되거나 prd_mart로, product_id로 join이 안되는 경우 추출.  
select count(*)
from up_mart a 
left outer join user_mart b
on a.user_id = b.user_id
left outer join prd_mart c
on a.product_id = c.product_id
where (b.user_id is null or c.product_id is null)

count(1)
3


In [0]:
%sql
select * from aisles where aisle_id='Blunted'
/* 
select * from products a where product_id = 6816
select * from aisles where aisle_id='Blunted' 
*/

aisle_id,aisle


In [0]:
%sql
-- 현재까지 만들어진 테이블의 건수 조사 
select 'user_mart count' as gubun, count(*) from user_mart
union all
select 'prd_mart count' as gubun, count(*) from prd_mart
union all
select 'up_mart count' as gubun, count(*) from up_mart

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: `default`.`up_mart` is not a Delta table.
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaTableException(DeltaErrors.scala:428)
	at com.databricks.sql.transaction.tahoe.catalog.DeltaTableV2.toBaseRelation(DeltaTableV2.scala:185)
	at com.databricks.sql.transaction.tahoe.DeltaRelation$.$anonfun$fromV2Relation$1(DeltaAnalysis.scala:778)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordFrameProfile(DeltaLogging.scala:171)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordFrameProfile$(DeltaLogging.scala:169)
	at com.databricks.sql.transaction.tahoe.DeltaRelation$.recordFrameProfile(DeltaAnalysis.scala:765)
	at com.databricks.sql.transaction.tahoe.DeltaRelation$.fromV2Relation(DeltaAnalysis.scala:777)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:222)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:79)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:171)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:167)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:171)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:324)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:169)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:165)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:30)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$4(AnalysisHelper.scala:176)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1226)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren$(TreeNode.scala:1225)
	at org.apache.spark.sql.catalyst.plans.logical.UnaryNode.mapChildren(LogicalPlan.scala:188)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:176)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:324)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:169)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:165)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:30)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$4(AnalysisHelper.scala:176)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1226)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren$(TreeNode.scala:1225)
	at org.apache.spark.sql.catalyst.plans.logical.UnaryNode.mapChildren(LogicalPlan.scala:188)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:176)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:324)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:169)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:165)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPla

#### 현재까지 만든 prd_mart, user_mart, up_mart를 결합하여 data_mart 생성. 
* 생성된 data_mart는 up_mart를 기준으로 prd_mart, user_mart를 조인하여 상품 분석속성, 사용자 분석속성을 결합.

In [0]:
%sql
describe up_mart

col_name,data_type,comment
user_id,int,
product_id,bigint,
up_cnt,bigint,
up_reord_cnt,bigint,
up_no_reord_cnt,bigint,
up_reoredered_avg,double,
up_max_ord_num,int,
up_min_ord_num,int,
up_avg_cart,double,
up_avg_prior_days,double,


In [0]:
print(spark.sql("select * from up_mart").columns)
print(spark.sql("select * from user_mart").columns)
print(spark.sql("select * from prd_mart").columns)

['user_id', 'product_id', 'up_cnt', 'up_reord_cnt', 'up_no_reord_cnt', 'up_reoredered_avg', 'up_max_ord_num', 'up_min_ord_num', 'up_avg_cart', 'up_avg_prior_days', 'up_max_prior_days', 'up_min_prior_days', 'up_avg_ord_dow', 'up_avg_ord_hour', 'up_usr_ratio', 'up_usr_reord_ratio', 'usr_reord_cnt', 'up_usr_ord_num_diff']
['user_id', 'usr_total_cnt', 'prd_uq_cnt', 'order_uq_cnt', 'usr_avg_prd_cnt', 'usr_avg_uq_prd_cnt', 'usr_uq_prd_ratio', 'usr_reord_cnt', 'usr_no_reord_cnt', 'usr_reordered_avg', 'usr_avg_prior_days', 'usr_max_prior_days', 'usr_min_prior_days', 'usr_avg_order_dow', 'usr_avg_order_hour_of_day', 'usr_max_order_number', 'order_id', 'eval_set', 'days_since_prior_order']
['product_id', 'prd_reordered_cnt', 'prd_no_reordered_cnt', 'prd_avg_reordered', 'prd_unq_usr_cnt', 'prd_total_cnt', 'prd_usr_ratio', 'prd_avg_prior_days', 'prd_min_prior_days', 'prd_max_prior_days', 'aisle_distinct_usr_cnt', 'aisle_total_cnt', 'aisle_usr_ratio', 'usr_ratio_diff']


In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/data_mart

res14: Boolean = false

In [0]:
%sql
drop table if exists data_mart;

-- up_mart에 user_mart를 user_id로 조인, prd_mart는 product_id로 조인하여 개별 xxx_mart테이블의 속성들을 취합하여 data_mart 테이블 생성. 약 4분정도 걸림. 
create table data_mart
as
select 
  -- up_mart 컬럼 
  a.user_id, a.product_id, b.order_id -- 테스트 데이터 예측 데이터 제출을 위해서 order_id가 필요함. 
  , up_cnt, up_reord_cnt, up_no_reord_cnt, up_reoredered_avg, up_max_ord_num, up_min_ord_num, up_avg_cart, up_avg_prior_days, up_max_prior_days
  , up_min_prior_days, up_avg_ord_dow, up_avg_ord_hour, up_usr_ratio, up_usr_reord_ratio, up_usr_ord_num_diff
  -- user_mart 컬럼, eval_set에 train과 test용 데이터(사용자)구분
  , usr_total_cnt, prd_uq_cnt, order_uq_cnt, usr_avg_prd_cnt, usr_avg_uq_prd_cnt, usr_uq_prd_ratio, a.usr_reord_cnt, usr_no_reord_cnt, usr_reordered_avg, usr_avg_prior_days
  , usr_max_prior_days, usr_min_prior_days, usr_avg_order_dow, usr_avg_order_hour_of_day, usr_max_order_number, eval_set, days_since_prior_order
  -- prd_mart 컬럼
  , prd_reordered_cnt, prd_no_reordered_cnt, prd_avg_reordered, prd_unq_usr_cnt, prd_total_cnt, prd_usr_ratio, prd_avg_prior_days, prd_min_prior_days, prd_max_prior_days
  , aisle_distinct_usr_cnt, aisle_total_cnt, aisle_usr_ratio, usr_ratio_diff
from up_mart a, user_mart b, prd_mart c
where a.user_id = b.user_id and a.product_id = c.product_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- 현재까지 생성된 테이블의 건수 조사. data_mart는 up_mart와 동일 건수 - 3 
select 'data_mart count' as gubun, count(*) from data_mart
union all 
select 'user_mart count' as gubun, count(*) from user_mart
union all
select 'prd_mart count' as gubun, count(*) from prd_mart
union all
select 'up_mart count' as gubun, count(*) from up_mart

gubun,count(1)
data_mart count,13307950
user_mart count,206209
prd_mart count,49676
up_mart count,13307953


In [0]:
%sql
select * from data_mart limit 10

user_id,product_id,order_id,up_cnt,up_reord_cnt,up_no_reord_cnt,up_reoredered_avg,up_max_ord_num,up_min_ord_num,up_avg_cart,up_avg_prior_days,up_max_prior_days,up_min_prior_days,up_avg_ord_dow,up_avg_ord_hour,up_usr_ratio,up_usr_reord_ratio,up_usr_ord_num_diff,usr_total_cnt,prd_uq_cnt,order_uq_cnt,usr_avg_prd_cnt,usr_avg_uq_prd_cnt,usr_uq_prd_ratio,usr_reord_cnt,usr_no_reord_cnt,usr_reordered_avg,usr_avg_prior_days,usr_max_prior_days,usr_min_prior_days,usr_avg_order_dow,usr_avg_order_hour_of_day,usr_max_order_number,eval_set,days_since_prior_order,prd_reordered_cnt,prd_no_reordered_cnt,prd_avg_reordered,prd_unq_usr_cnt,prd_total_cnt,prd_usr_ratio,prd_avg_prior_days,prd_min_prior_days,prd_max_prior_days,aisle_distinct_usr_cnt,aisle_total_cnt,aisle_usr_ratio,usr_ratio_diff
4041,34063,730911,8,7,1,0.875,27,4,4.875,15.5,30.0,7.0,3.25,13.0,0.02359882005899705,0.04242424242424243,0,339,174,27,12.555555555555555,1.9482758620689655,0.5132743362831859,165,174,0.48672566371681414,12.615615615615615,30.0,2.0,3.274336283185841,11.050147492625369,27,test,20.0,1057,738,0.588857938718663,738,1795,0.41114206128133707,12.03411131059246,0.0,30.0,46636,207075,0.22521308704575638,0.1859289742355807
99471,14917,2560214,11,10,1,0.9090909090909091,17,1,6.818181818181818,7.5,15.0,5.0,3.0,13.0,0.031161473087818695,0.0423728813559322,2,353,117,19,18.57894736842105,3.017094017094017,0.3314447592067989,236,117,0.6685552407932012,10.475609756097562,30.0,5.0,2.903682719546742,13.025495750708215,19,test,19.0,2411,1055,0.6956145412579342,1055,3466,0.3043854587420658,9.70420829548895,0.0,30.0,124393,1452343,0.08564987747384743,0.21873558126821835
7440,18770,3078569,16,15,1,0.9375,99,62,7.875,3.875,15.0,1.0,2.625,13.75,0.013311148086522463,0.015544041450777202,0,1202,237,99,12.141414141414142,5.071729957805907,0.19717138103161397,965,237,0.802828618968386,3.687925170068027,18.0,0.0,2.9525790349417638,14.537437603993345,99,test,1.0,1177,897,0.5675024108003858,897,2074,0.4324975891996143,10.902051282051282,0.0,30.0,78030,395130,0.1974793106066358,0.2350182785929785
34858,21174,3009068,6,5,1,0.8333333333333334,42,2,5.666666666666667,5.166666666666667,15.0,0.0,2.5,13.666666666666666,0.008695652173913044,0.011574074074074073,5,690,258,47,14.680851063829786,2.6744186046511627,0.3739130434782609,432,258,0.6260869565217392,7.616191904047976,22.0,0.0,2.1333333333333333,13.7,47,train,5.0,6852,5509,0.5543240838119893,5509,12361,0.44567591618801067,11.14011466296039,0.0,30.0,159418,1765313,0.09030579846180252,0.35537011772620813
16181,35168,1675807,5,4,1,0.8,14,5,10.0,15.8,30.0,6.0,1.2,12.4,0.013774104683195593,0.01568627450980392,5,363,108,19,19.105263157894736,3.361111111111111,0.2975206611570248,255,108,0.7024793388429752,13.746438746438747,30.0,5.0,1.578512396694215,13.110192837465565,19,train,30.0,3250,2618,0.5538513974096796,2618,5868,0.4461486025903204,11.240999265246142,0.0,30.0,57255,193297,0.2962022173132537,0.14994638527706672
153133,21137,1769478,2,1,1,0.5,17,13,24.5,23.0,30.0,16.0,4.0,19.5,0.008658008658008658,0.0070921985815602835,0,231,90,17,13.588235294117647,2.566666666666667,0.38961038961038963,141,90,0.6103896103896104,14.748815165876778,30.0,1.0,3.0043290043290045,14.80952380952381,17,test,7.0,205845,58838,0.7777038948477991,58838,264683,0.22229610515220094,10.03712850345864,0.0,30.0,177141,3642188,0.04863587491914201,0.17366023023305893
29916,26209,2199322,3,2,1,0.6666666666666666,7,3,8.333333333333334,20.666666666666668,30.0,7.0,3.0,11.333333333333334,0.03225806451612903,0.047619047619047616,0,93,51,7,13.285714285714286,1.8235294117647058,0.5483870967741935,42,51,0.45161290322580644,19.513157894736842,30.0,7.0,2.7419354838709675,12.118279569892474,7,train,30.0,95768,44859,0.6810072034531064,44859,140627,0.31899279654689355,11.135485385786456,0.0,30.0,177141,3642188,0.04863587491914201,0.27035692162775155
87185,44570,3278341,5,4,1,0.8,11,4,8.8,12.2,30.0,0.0,1.8,16.4,0.017921146953405017,0.030534351145038167,3,279,148,14,19.928571428571427,1.8851351351

### 학습과 테스트용 데이터 세트 생성. 
* order_products_train.csv(trains 테이블)는  train용으로 reordered label 값이 주어져 있음.
* trains 테이블의 pk는 order_id + product_id 이지만 실제로는 1개의 user_id에 1개의 order_id만 할당되므로 user_id + product_id로 unique함. 
* trains 테이블과 orders 테이블을 조인하여 user_id를 가져오는 order_trains_prods 테이블 생성. 
* order_trains_prods 테이블을 기준으로 data_mart에서 생성한 속성을 붙이려고 두개의 테이블을 user_id + product_id로 조인(order_trains_prods 레프트 아우터 조인)하면 많은 데이터가 조인되지 않음.  
* 조인되지 않을 경우에 data_mart에서 생성한 속성을 사용할 수 없음. 
* data_mart를 기준으로 order_trains_prods를 조인(data_mart 레프트 아우터 조인)하여 label값인 reordered를 설정하고 조인되지 않는 경우 reordered를 0으로 설정.

In [0]:
%sql
select * from trains limit 10

order_id,product_id,add_to_cart_order,reordered
1,49302,1,1
1,11109,2,1
1,10246,3,0
1,49683,4,0
1,43633,5,1
1,13176,6,0
1,47209,7,0
1,22035,8,1
36,39612,1,0
36,19660,2,1


In [0]:
%sql
--1384617
select count(*) from trains

count(1)
1384617


In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/order_trains_prods

res15: Boolean = false

In [0]:
%sql
drop table if exists order_trains_prods;
-- order_products_train 데이터에(trains 테이블)에 user_id를 얻기 위해서 orders 테이블과 조인
-- 해당 테이블은 kaggle에서 train 용으로 제공었지만, 많은 속성(feature)로 만들어진 data_mart 테이블에 비해 적은 속성을 가지고 있음. 
create table order_trains_prods
as
select a.order_id, a.product_id, a.reordered
  , b.user_id
from trains a, orders b
where a.order_id = b.order_id

num_affected_rows,num_inserted_rows
